In [1]:
import os
from glob import glob
import dask
import xarray as xr
import cartopy.crs as ccrs
from cmocean import cm
import numpy as np
import xgcm
from scipy.io import loadmat

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/docrep/__init__.py:413: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  s = dedents(s)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/docrep/__init__.py:342: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  s = dedents('\n' + '\n'.join(lines[first:]))


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)

w = cluster.start_workers(5)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/docrep/__init__.py:342: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  s = dedents('\n' + '\n'.join(lines[first:]))


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://10.148.0.9:56699 Dashboard: http://10.148.0.9:8787/status,Cluster Workers: 6 Cores: 24 Memory: 300.00 GB


# Read rechunked data

In [6]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

ds_u = xr.open_zarr(out_dir_zarr+'U_total_rechunk.zarr')
ds_v = xr.open_zarr(out_dir_zarr+'V_total_rechunk.zarr')

In [7]:
ds_u_surface = ds_u.isel(depth=0)
ds_v_surface = ds_v.isel(depth=0)

In [8]:
ds_u_surface = ds_u_surface.chunk({'time': 9415, 'lat': 201, 'lon_g': 177})
ds_v_surface = ds_v_surface.chunk({'time': 9415, 'lat_g': 201, 'lon': 177})

In [9]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

file_out = out_dir_zarr+'U_surface_rechunk.nc'
%time ds_u_surface.to_netcdf(file_out, mode='w')     

file_out = out_dir_zarr+'V_surface_rechunk.nc'
%time ds_v_surface.to_netcdf(file_out, mode='w')   

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 236 ms, sys: 24 ms, total: 260 ms
Wall time: 4.57 s


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 160 ms, sys: 24 ms, total: 184 ms
Wall time: 2.58 s


# Surface KE

In [5]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

ds_u = xr.open_dataset(out_dir_zarr+'U_surface_rechunk.nc')
ds_v = xr.open_dataset(out_dir_zarr+'V_surface_rechunk.nc')
grid = xr.open_dataset(out_dir_zarr+'Grid.nc', decode_coords=True)

ds = xr.merge([ds_u,ds_v,grid])
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177, time: 9415)
Coordinates:
  * lat      (lat) float64 47.4 47.42 47.43 47.44 47.46 47.47 47.48 47.5 ...
  * lon_g    (lon_g) float64 -18.04 -18.02 -18.0 -17.98 -17.96 -17.94 -17.92 ...
    depth    float64 0.5
  * time     (time) float64 0.0 0.04167 0.08333 0.125 0.1667 0.2083 0.25 ...
  * lat_g    (lat_g) float64 47.4 47.41 47.42 47.44 47.45 47.46 47.48 47.49 ...
  * lon      (lon) float64 -18.03 -18.01 -17.99 -17.97 -17.95 -17.93 -17.91 ...
    XC       (lat, lon) float32 ...
    YC       (lat, lon) float32 ...
    XG       (lat_g, lon_g) float32 ...
    YG       (lat_g, lon_g) float32 ...
    CS       (lat, lon) float32 ...
    SN       (lat, lon) float32 ...
    Depth    (lat, lon) float32 ...
    dxC      (lat, lon_g) float32 ...
    dyC      (lat_g, lon) float32 ...
    dxG      (lat_g, lon) float32 ...
    dyG      (lat, lon_g) float32 ...
    rA       (lat, lon) float32 ...
    rAz      (lat_g, lon_g) floa

In [7]:
gridx = xgcm.Grid(ds, coords={'X':{'center':'lon', 'left':'lon_g'},
                              'Y':{'center':'lat', 'left':'lat_g'}})
print(gridx)

<xgcm.Grid>
X Axis (periodic):
  * center   lon (177) --> left
  * left     lon_g (177) --> center
Y Axis (periodic):
  * center   lat (201) --> left
  * left     lat_g (201) --> center


In [22]:
OND = slice(433,2640)
JFM = slice(2641,4824)
AMJ = slice(4825,7008)
JAS = slice(7009,9216)

In [23]:
U = ds.U.isel(time=OND)
U = np.real(U).rename('U')

V = ds.V.isel(time=OND)
V = np.real(V).rename('V')

KE = (gridx.interp(ds['U'].isel(time=OND)**2,'X') + gridx.interp(ds['V'].isel(time=OND)**2,'Y'))/2 # KE
KE = np.real(KE).rename('KE')

ds_OND = xr.merge([U,V,KE]).drop(['depth','dxC','dyG','rAw','dyC','dxG','rAs'])
ds_OND

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177, time: 2207)
Coordinates:
  * lat      (lat) float64 47.4 47.42 47.43 47.44 47.46 47.47 47.48 47.5 ...
  * lon_g    (lon_g) float64 -18.04 -18.02 -18.0 -17.98 -17.96 -17.94 -17.92 ...
  * time     (time) float64 18.04 18.08 18.12 18.17 18.21 18.25 18.29 18.33 ...
  * lat_g    (lat_g) float64 47.4 47.41 47.42 47.44 47.45 47.46 47.48 47.49 ...
  * lon      (lon) float64 -18.03 -18.01 -17.99 -17.97 -17.95 -17.93 -17.91 ...
Data variables:
    U        (time, lat, lon_g) float32 -0.11759235 -0.13874727 -0.15098977 ...
    V        (time, lat_g, lon) float32 0.102351956 0.10284708 0.11202508 ...
    KE       (time, lat, lon) float32 0.012609762 0.015105447 0.015200166 ...

In [24]:
U = ds.U.isel(time=JFM)
U = np.real(U).rename('U')

V = ds.V.isel(time=JFM)
V = np.real(V).rename('V')

KE = (gridx.interp(ds['U'].isel(time=JFM)**2,'X') + gridx.interp(ds['V'].isel(time=JFM)**2,'Y'))/2 # KE
KE = np.real(KE).rename('KE')

ds_JFM = xr.merge([U,V,KE]).drop(['depth','dxC','dyG','rAw','dyC','dxG','rAs'])
ds_JFM

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177, time: 2183)
Coordinates:
  * lat      (lat) float64 47.4 47.42 47.43 47.44 47.46 47.47 47.48 47.5 ...
  * lon_g    (lon_g) float64 -18.04 -18.02 -18.0 -17.98 -17.96 -17.94 -17.92 ...
  * time     (time) float64 110.0 110.1 110.1 110.2 110.2 110.2 110.3 110.3 ...
  * lat_g    (lat_g) float64 47.4 47.41 47.42 47.44 47.45 47.46 47.48 47.49 ...
  * lon      (lon) float64 -18.03 -18.01 -17.99 -17.97 -17.95 -17.93 -17.91 ...
Data variables:
    U        (time, lat, lon_g) float32 0.06518597 0.0138738835 -0.03224002 ...
    V        (time, lat_g, lon) float32 -0.050177623 -0.02169811 0.020780176 ...
    KE       (time, lat, lon) float32 0.0019146921 0.00045890003 ...

In [25]:
U = ds.U.isel(time=AMJ)
U = np.real(U).rename('U')

V = ds.V.isel(time=AMJ)
V = np.real(V).rename('V')

KE = (gridx.interp(ds['U'].isel(time=AMJ)**2,'X') + gridx.interp(ds['V'].isel(time=AMJ)**2,'Y'))/2 # KE
KE = np.real(KE).rename('KE')

ds_AMJ = xr.merge([U,V,KE]).drop(['depth','dxC','dyG','rAw','dyC','dxG','rAs'])
ds_AMJ

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177, time: 2183)
Coordinates:
  * lat      (lat) float64 47.4 47.42 47.43 47.44 47.46 47.47 47.48 47.5 ...
  * lon_g    (lon_g) float64 -18.04 -18.02 -18.0 -17.98 -17.96 -17.94 -17.92 ...
  * time     (time) float64 201.0 201.1 201.1 201.2 201.2 201.2 201.3 201.3 ...
  * lat_g    (lat_g) float64 47.4 47.41 47.42 47.44 47.45 47.46 47.48 47.49 ...
  * lon      (lon) float64 -18.03 -18.01 -17.99 -17.97 -17.95 -17.93 -17.91 ...
Data variables:
    U        (time, lat, lon_g) float32 0.33727488 0.3330282 0.31301385 ...
    V        (time, lat_g, lon) float32 0.025622167 0.08200298 0.13494726 ...
    KE       (time, lat, lon) float32 0.057484288 0.057119038 0.054811202 ...

In [26]:
U = ds.U.isel(time=JAS)
U = np.real(U).rename('U')

V = ds.V.isel(time=JAS)
V = np.real(V).rename('V')

KE = (gridx.interp(ds['U'].isel(time=JAS)**2,'X') + gridx.interp(ds['V'].isel(time=JAS)**2,'Y'))/2 # KE
KE = np.real(KE).rename('KE')

ds_JAS = xr.merge([U,V,KE]).drop(['depth','dxC','dyG','rAw','dyC','dxG','rAs'])
ds_JAS

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177, time: 2207)
Coordinates:
  * lat      (lat) float64 47.4 47.42 47.43 47.44 47.46 47.47 47.48 47.5 ...
  * lon_g    (lon_g) float64 -18.04 -18.02 -18.0 -17.98 -17.96 -17.94 -17.92 ...
  * time     (time) float64 292.0 292.1 292.1 292.2 292.2 292.2 292.3 292.3 ...
  * lat_g    (lat_g) float64 47.4 47.41 47.42 47.44 47.45 47.46 47.48 47.49 ...
  * lon      (lon) float64 -18.03 -18.01 -17.99 -17.97 -17.95 -17.93 -17.91 ...
Data variables:
    U        (time, lat, lon_g) float32 0.10475846 0.139333 0.14304297 ...
    V        (time, lat_g, lon) float32 -0.312462 -0.32103947 -0.32232642 ...
    KE       (time, lat, lon) float32 0.056700964 0.06282061 0.06262124 ...

In [27]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

file_out = out_dir_zarr+'KE_U_V_surface_rechunk_OND.nc'
%time ds_OND.to_netcdf(file_out, mode='w')   

file_out = out_dir_zarr+'KE_U_V_surface_rechunk_JFM.nc'
%time ds_JFM.to_netcdf(file_out, mode='w')   

file_out = out_dir_zarr+'KE_U_V_surface_rechunk_AMJ.nc'
%time ds_AMJ.to_netcdf(file_out, mode='w')   

file_out = out_dir_zarr+'KE_U_V_surface_rechunk_JAS.nc'
%time ds_JAS.to_netcdf(file_out, mode='w')   


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 12 ms, sys: 256 ms, total: 268 ms
Wall time: 570 ms


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 0 ns, sys: 252 ms, total: 252 ms
Wall time: 373 ms


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 4 ms, sys: 248 ms, total: 252 ms
Wall time: 368 ms


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 8 ms, sys: 256 ms, total: 264 ms
Wall time: 369 ms


In [11]:
cluster.scheduler.close()

<Future finished result=None>